In [32]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
final_data1 = load_result_dataset(pn1, pn2)
df1 = pd.DataFrame(final_data1)
df1['ft_strategy'] = 'full_fine_tuning_50epochs'

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
final_data2 = load_result_dataset(pn1, pn2)
df2 = pd.DataFrame(final_data2)
df2['ft_strategy'] = 'full_fine_tuning_5epochs'

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
final_data3 = load_result_dataset(pn1, pn2)
df3 = pd.DataFrame(final_data3)
df3['ft_strategy'] = 'linearprobe_50epochs'


../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [85]:
import plotly.graph_objects as go
import pandas as pd

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import spearmanr
import numpy as np
from numpy.linalg import LinAlgError
from scipy.stats import linregress

import pandas as pd
from scipy.stats import spearmanr, pearsonr
from process_database import process_grouped_df, process_rankings,global_grouped_dataset

df = pd.concat([ df1,df2,df3 ], axis=0, ignore_index=True) 
grouped_df = global_grouped_dataset(df,)
grouped_df = process_rankings(grouped_df)

grouped_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped_df.columns]
grouped_df = grouped_df.reset_index()

grouped_df = grouped_df[["ft_strategy", "model_type", "model_size", "backbone", "loss_function", "TOTAL_score_sum"]]

results = []
for strategy in [ "full_fine_tuning_50epochs", "full_fine_tuning_5epochs", "linearprobe_50epochs" ]:
    for size in ["small", "medium", "large"]:

        res = grouped_df[
            (grouped_df.ft_strategy == strategy) &
            (grouped_df.model_size == size)
        ]

        q = res["TOTAL_score_sum"]

        # Compute quantiles
        quantiles = q.quantile([0.25, 0.5, 0.75, 0.90])
        labels = ['25%', '50%', '75%', '90%']
        counts = {}
        for label, thresh in zip(labels, quantiles):
            counts[label] = (q <= thresh).sum()
        # counts['min'] = 1
        # counts['max'] = 1

        stats = q.quantile([0.25, 0.5, 0.75, 0.90, ]).rename(index={0.25: '25%', 
                                                                            0.5: '50%', 
                                                                            0.75: '75%',
                                                                            0.90: '90%',})

        # Append min and max
        min_val = q.min()
        max_val = q.max()

        q_extended = pd.concat([stats, pd.Series({"min": min_val, "max": max_val})]).to_dict()
        q_extended['strategy'] = strategy
        q_extended["size"] = size
        for key, count in counts.items():
            q_extended[f"{key}_count"] = count
        results.append( q_extended )


import pandas as pd
import plotly.express as px

# Create DataFrame
df = pd.DataFrame(results)

# Melt the DataFrame
df_long = df.melt(id_vars=['strategy', 'size'], var_name='percentile', value_name='value')

# Define proper order
ordered_percentiles = ['min', '25%', '50%', '75%', '90%', 'max']
df_long['percentile'] = pd.Categorical(df_long['percentile'], categories=ordered_percentiles, ordered=True)

def get_count(row):
    colname = f"{row['percentile']}_count"
    # Get the count from the original df based on strategy & size
    subset = df[(df['strategy'] == row['strategy']) & (df['size'] == row['size'])]
    if colname in subset.columns:
        return int(subset[colname].values[0])
    else:
        return 'N/A'

df_long["hover_text"] = df_long.apply(
    lambda row: f"{row['strategy']} – {row['size']}<br>{row['percentile']} = {row['value']:.2f}<br>n={get_count(row)}",
    axis=1
)

# ✅ Sort so lines connect in correct order
df_long = df_long.sort_values(by=['strategy', 'size', 'percentile'])

# Plot
fig = px.line(
    df_long,
    x='percentile',
    y='value',
    color='strategy',
    line_dash='size',
    markers=True,
    labels={'value': 'Total Accuracy Score',
            'percentile': 'Quantile of total score',},
    # title='Score Percentiles by Strategy and Model Size',
    category_orders={'percentile': ordered_percentiles}
)

import plotly.graph_objects as go

for i, row in df_long.iterrows():
    count_key = f"{row['percentile']}_count"
    try:
        count = df[
            (df['strategy'] == row['strategy']) &
            (df['size'] == row['size'])
        ][count_key].values[0]
        fig.add_annotation(
            x=row['percentile'],
            y=row['value'],
            text=f"n={count}",
            showarrow=False,
            font=dict(size=8),
            xanchor="left",
            yanchor="bottom"
        )
    except:
        pass

fig.update_layout(yaxis=dict(range=[0, 18]),)
fig.update_layout(
    width=450,
    height=400,
    margin=dict(l=0, r=0, t=0, b=0),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="lightgrey"),
    yaxis=dict(showgrid=True, gridcolor="lightgrey"),
    legend_title_text="Strategy / Architecture Size",

    legend=dict(
    x=0.05,
    y=1,
    xanchor='left',
    yanchor='top',
    font=dict(size=10),  # Increase font size
    bgcolor="rgba(255,255,255,0.5)",
    bordercolor="lightgrey",
    borderwidth=1,
    itemsizing="constant"  # 👈 Key to reduce spacing
)
)

fig.show()
fig.write_image("./paper_figures/quantile_perf_improve.png", scale=3  )


    

In [5]:
import pandas as pd
import plotly.express as px


custom_palette = ["#E69F00", "#56B4E9", "#009E73"]  # choose a set above

# ── 1.  Rank backbones by their aggregate TOTAL_score_sum ───────────────────────
backbone_order = (
    sub_df.groupby("backbone")["TOTAL_score_sum"]
          .sum()                       # sum over the 6 settings
          .sort_values(ascending=False)
          .index                       # → ordered list of backbones
          .tolist()
)

# Make the column an ordered categorical so Plotly respects the ranking
sub_df["backbone"] = pd.Categorical(
    sub_df["backbone"],
    categories=backbone_order,
    ordered=True
)

# ── 2.  Custom colours / markers ───────────────────────────────────────────────
color_map = {
    "full_fine_tuning_50epochs": custom_palette[0],#"orange",
    "full_fine_tuning_5epochs":  custom_palette[1],#"purple",
    "linearprobe_50epochs": custom_palette[2]     #"green",
}
symbol_map = {"TRADES_v2": "star", "CLASSIC_AT": "square"}

# ── 3.  Plot ───────────────────────────────────────────────────────────────────
fig = px.scatter(
    sub_df,
    width=1500, 
    height=1000,
    color_discrete_sequence=custom_palette,
    x= "backbone",
    y="TOTAL_score_sum",
    color="ft_strategy",
    symbol="loss_function",
    color_discrete_map=color_map,
    symbol_map=symbol_map,
    hover_data=["backbone", "ft_strategy", "loss_function", "TOTAL_score_sum"],
)

fig.update_layout(
    paper_bgcolor="white",   # outside the plotting area
    plot_bgcolor="white",    # inside the plotting area

    xaxis=dict(
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=False       # optional: hide the heavy axis line
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=False
    )
)

fig.update_layout(yaxis_title="Aggregate Accuracy Score")

fig.update_layout(showlegend=True)   # keep or turn off if it clutters
fig.show()

fig.write_image("./global_plot.png", scale=3  )                # upscale for higher DPI (1 = default)



/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_74754/3436293082.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["backbone"] = pd.Categorical(


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import spearmanr
import numpy as np
from numpy.linalg import LinAlgError
from scipy.stats import linregress

import pandas as pd
from scipy.stats import spearmanr, pearsonr
from process_database import process_grouped_df, process_rankings,global_grouped_dataset


grouped_df = global_grouped_dataset(df,)
grouped_df = process_rankings(grouped_df)
# Flatten the columns
grouped_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped_df.columns]
grouped_df = grouped_df.reset_index()
sub_df = grouped_df[["ft_strategy", "model_type", "model_size", "backbone", "loss_function", "TOTAL_score_sum"]]

# ── 1.  Reshape: one row per backbone × loss_function, one column per project
pivot_df = sub_df.pivot_table(
    index=["backbone", "loss_function",  "model_type", "model_size",],
    columns="ft_strategy",
    values="TOTAL_score_sum"
).reset_index()

symbol_map = {"TRADES_v2": "diamond", "CLASSIC_AT": "star"}

color_palette = ["#E69F00", "#56B4E9",  "#009E73",  ] # "#009E73", "#F0E442", "#0072B2","#CC79A7", "#D55E00",
unique_sizes = pivot_df["model_size"].unique()
color_map = {size: color_palette[i % len(color_palette)] for i, size in enumerate(unique_sizes)}


for (x1,x2) in [ ("full_fine_tuning_5epochs", "full_fine_tuning_50epochs"), ("linearprobe_50epochs", "full_fine_tuning_50epochs") ]:

    # Global correlation
    pivot_df = pivot_df.fillna(0)
    x = pivot_df[x1]
    y = pivot_df[x2]
    r_global, p_global = spearmanr(x, y)
    try:
        # m_global, b_global = np.polyfit(x, y, 1)
        m_global, b_global, r, p, _ = linregress(x, y)
    except Exception as e:
        print(f"Warning: polyfit failed with error: {e}")
        m_global, b_global = 0, y.mean()  # or any safe fallback
    x_range = np.linspace(x.min(), x.max(), 100)

    # Base scatter plot
    fig = px.scatter(
        pivot_df,
        x=x1,
        y=x2,
        color="model_size",
        symbol="loss_function",
        symbol_map=symbol_map,
        color_discrete_map=color_map,
        labels={
            "full_fine_tuning_5epochs": "Fine-tuning (5 epochs)",
            "full_fine_tuning_50epochs": "Fine-tuning (50 epochs)",
            "linearprobe_50epochs": "Linear-probing (50 epochs)"
        },
        #title="Fine-Tuning Correlation (5 vs 50 Epochs)"
    )

    # Global black line
    fig.add_trace(go.Scatter(
        x=x_range,
        y=m_global * x_range + b_global,
        mode="lines",
        line=dict(color="black", dash="solid", width=2),
        name=f"Global (r={r_global:.2f}, p={p_global:.3f})"
    ))



    # Add per-group lines
    for model_size in pivot_df["model_size"].unique():
        for loss_function, style in zip(["TRADES_v2", "CLASSIC_AT"], ["solid", "dot"]):
            subset = pivot_df[(pivot_df["model_size"] == model_size) & (pivot_df["loss_function"] == loss_function)]
            if len(subset) >= 2:
                xg, yg = subset[x1], subset[x2]
                rg, pg = spearmanr(xg, yg)

                try:
                    mg, bg, r, p, _ = linregress(xg, yg)
                except Exception as e:
                    print(f"Warning: polyfit failed with error: {e}")
                    mg, bg = 0, yg.mean()  # or any safe fallback
                    
                fig.add_trace(go.Scatter(
                    x=x_range,
                    y=mg * x_range + bg,
                    mode="lines",
                    line=dict(color=color_map[model_size], dash=style),
                    name=f"{model_size} – {loss_function} (r={rg:.2f}, p={pg:.3f})"
                ))

    fig.update_layout(yaxis=dict(range=[0, 20]),)
    fig.update_layout(xaxis=dict(range=[-0.25, 8.25]),)
    # Layout tweaks
    fig.update_traces(marker=dict(size=8, line=dict(width=0, color='DarkSlateGrey')))
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),  # remove all outer margins
        width=800, #if x1=='full_fine_tuning_5epochs' else 488 ,    # in pixels (~2.5 inches at 96 DPI)
        height=400,    # in pixels (~1.875 inches)
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(showgrid=True, gridcolor="lightgrey"),
        yaxis=dict(showgrid=True, gridcolor="lightgrey"),
        legend_title_text="Legend",
        #showlegend = True if x1=='full_fine_tuning_5epochs' else False,

                    # Legend inside figure
        legend=dict(
                font=dict(size=12),
                y=0.85,           # optional: near the top
                bgcolor="rgba(255,255,255,1)",
                bordercolor="lightgrey",
                borderwidth=1
            )
    )

    fig.show()
    fig.write_image("./paper_figures/low_cost_proxy_{}.png".format(x1,x2), scale=3  )



In [ ]:
import pandas as pd
from scipy.stats import fisher_exact
from process_database import process_grouped_df, process_rankings
from load_results import load_result_dataset

# Collector for odds ratios and p-values
results = []

for (name, pn1, pn2) in [

    ('Full fine-tuning (50 epochs)','full_fine_tuning_50epochs_edge_paper_final2', 'full_fine_tuning_50epochs_paper_final2'),
    ('Full fine-tuning (5 epochs)', 'full_fine_tuning_5epochs_edge_article1', 'full_fine_tuning_5epochs_article1'),
    ('Linear probing (50 epochs)', 'linearprobe_50epochs_edge_paper_final2', 'linearprobe_50epochs_paper_final2') ]:
    
    final_data = load_result_dataset(pn1, pn2)

    for size_id, size in [(0, 'small'), (1, 'medium'), (2, 'large')]:

        grouped_df = process_grouped_df(final_data, size_id)
        grouped_df = process_rankings(grouped_df)

        odds_df = grouped_df['TOTAL'].reset_index()

        # Unique ID for observation
        index_cols = ['backbone', 'pre_training_strategy', 'model_type']
        score_col = "score_sum"

        pivot = odds_df.pivot_table(
            index=index_cols,
            columns="loss_function",
            values=score_col
        ).dropna()

        # Pairwise TRADES vs CLASSIC
        pivot["winner"] = pivot.apply(
            lambda row: "TRADES" if row["TRADES_v2"] > row["CLASSIC_AT"]
            else "CLASSIC" if row["TRADES_v2"] < row["CLASSIC_AT"]
            else "TIE",
            axis=1
        )

        wins = pivot[pivot["winner"] != "TIE"]
        trades_wins = (wins["winner"] == "TRADES").sum()
        classic_wins = (wins["winner"] == "CLASSIC").sum()

        table = [[trades_wins, classic_wins], [classic_wins, trades_wins]]
        odds_ratio, p_value = fisher_exact(table)

        # Collect result
        results.append({
            "protocol": name,
            "size": size,
            "trades_wins": trades_wins,
            "classic_wins": classic_wins,
            "odds_ratio": odds_ratio,
            "p_value": p_value
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [ ]:
import plotly.express as px
import pandas as pd

# Add sample size column
results_df["sample_size"] = results_df["trades_wins"] + results_df["classic_wins"]

# Format text label with odds ratio, p-value, and sample size
results_df["text_label"] = results_df.apply(
    lambda row: (
        f'{int(row["odds_ratio"])}' if row["odds_ratio"] == int(row["odds_ratio"]) 
        else f'{row["odds_ratio"]:.2f}'
    ) + f'<br>(p={row["p_value"]:.3f})<br>n={row["sample_size"]}',
    axis=1
)
# Plotly bar chart
fig = px.bar(
    results_df,
    x="size",
    y="odds_ratio",
    color="protocol",
    barmode="group",
    text="text_label",
    labels={"size": "Model Size", "odds_ratio": "Odds Ratio"},
)

# Update traces for text styling
fig.update_traces(
    textposition="outside",
    textangle=-90,
    textfont=dict(size=10),
    cliponaxis=False
)


fig.update_layout(
            margin=dict(l=0, r=0, t=0, b=0),  # remove all outer margins
            width=450,    # in pixels (~2.5 inches at 96 DPI)
            height=400,    # in pixels (~1.875 inches)
            yaxis_title="Odds Ratio",
            xaxis_title="Model Size",            
            # White background
            plot_bgcolor="white",
            paper_bgcolor="white",
            
            # Light grey grid
            xaxis=dict(showgrid=False, gridcolor="lightgrey"),
            yaxis=dict(showgrid=True, gridcolor="lightgrey"),
            uniformtext_minsize=8,
            uniformtext_mode='show',
            
            # Legend inside figure
            legend=dict(
                font=dict(size=12),
                yanchor="top",
                xanchor="left",   # anchor point for x position
                x=0.05,           # move legend to the left side
                y=1.15,           # optional: near the top
                bgcolor="rgba(255,255,255,1)",
                bordercolor="lightgrey",
                borderwidth=1
            )
        )


fig.show()
fig.write_image("./paper_figures/oddsratio_TRADES_outperforms.png", scale=3  )
